In [1]:
# from gym.envs.registration import register
# register(
#     id='FrozenLakeNotSlippery-v0',
#     entry_point='gym.envs.toy_text:FrozenLakeEnv',
#     kwargs={'map_name' : '4x4', 'is_slippery': False},
#     max_episode_steps=100,
#     reward_threshold=0.8196, # optimum = .8196, changing this seems have no influence
# )

In [2]:
import gym
import numpy as np

In [26]:
# env = gym.make('FrozenLake-v0', is_slippery=False)
env = gym.make('FrozenLake-v0')
# env = gym.make("FrozenLakeNotSlippery-v0")

In [27]:
print("observation_space: ", env.observation_space.n)
print("action_space: ", env.action_space.n)
env.P

observation_space:  16
action_space:  4


{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [28]:

def value_iteration(env, gamma=1.0):
    value_table = np.zeros(env.observation_space.n)
    policy = np.zeros(env.observation_space.n)
    no_of_iterations = 100000
    threshold = 1e-20
    
    for i in range(no_of_iterations):
        updated_value_table = value_table.copy()

        for state in range(env.observation_space.n):
            Q_value = []
            for action in range(env.action_space.n):
                next_states_rewards = []
                for next_sr in env.P[state][action]:
                    trans_prob, next_state, reward_prob, _ = next_sr
                    next_states_rewards.append((trans_prob* (reward_prob+gamma*updated_value_table[next_state])))
                Q_value.append(np.sum(next_states_rewards))
            value_table[state] = max(Q_value)
            policy[state] = np.argmax(Q_value) # this is done by mausd. actullay extract_policy function done same thing. 
            #print(Q_value)
#         print(value_table)
#         print()
        if(np.sum(np.fabs(updated_value_table-value_table))<threshold):
            print("Value-iteration converged at iteration# %d."%(i+1))
            break
            
    return value_table, policy

            

In [29]:
optimal_value_function, optimal_policy = value_iteration(env)
print( optimal_policy)
print(optimal_value_function)

Value-iteration converged at iteration# 1373.
[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]
[0.82352941 0.82352941 0.82352941 0.82352941 0.82352941 0.
 0.52941176 0.         0.82352941 0.82352941 0.76470588 0.
 0.         0.88235294 0.94117647 0.        ]


In [30]:
def extract_policy(value_table, gamma = 1.0):
    policy = np.zeros(env.observation_space.n)
    for state in range(env.observation_space.n):
        Q_table = np.zeros(env.action_space.n)
        for action in range(env.action_space.n):
            for next_sr in env.P[state][action]:
                trans_prob, next_state, reward_prob, _ = next_sr
                Q_table[action]+=(trans_prob*(reward_prob+gamma*value_table[next_state]))
        policy[state] = np.argmax(Q_table)
    return  policy

In [31]:
optimal_policy_function = extract_policy(optimal_value_function)
print(optimal_policy_function)

[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]


In [9]:
for state in range(env.observation_space.n):
    for action in range(env.action_space.n):
        print("state: {}, action: {}".format(state, action))
        print(env.P[state][action])
        print()

state: 0, action: 0
[(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False)]

state: 0, action: 1
[(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False)]

state: 0, action: 2
[(0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False)]

state: 0, action: 3
[(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False)]

state: 1, action: 0
[(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 5, 0.0, True)]

state: 1, action: 1
[(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 5, 0.0, True), (0.3333333333333333, 2, 0.0, False)]

state: 1, action: 2
[(0.3333333333333333, 5, 0.0, True), (0.3333333333333333, 2, 0.0, False), (0.3333333333333333, 1, 0.0, False)]

state: 1, action: 3
[(0.3333333333333333, 2, 0.0, False), (0.33333333333

In [34]:
def compute_value_function(policy, gamma = 1.0):
    value_table = np.zeros(env.nS)
    threshold = 1e-10
    while True:
        updated_value_table = value_table.copy()
        for state in range(env.nS):
            action = policy[state]
                
            value_table[state] = sum([trans_prob * (reward_prob + gamma *updated_value_table[next_state])\
                                      for trans_prob, next_state, reward_prob, _ in env.P[state][action]])

        #print(np.sum(np.fabs(updated_value_table-value_table)))
        if(np.sum(np.fabs(updated_value_table-value_table))<=threshold):
            break
    return value_table
                                      

In [35]:
print(env.nS)

16


In [36]:
random_policy = np.zeros(env.nS)
optimize_value_table = compute_value_function(random_policy)

In [37]:
def policy_iteration(env,gamma = 1.0):
    random_policy = np.zeros(env.observation_space.n)
    no_of_iterations = 200
    for i in range(no_of_iterations):
        new_value_function = compute_value_function(random_policy, gamma)
        new_policy = extract_policy(new_value_function, gamma)
        if(np.all(random_policy==new_policy)):
            print('Policy-Iteration converged at step %d.' %(i+1))
            break
        if(i%10==0):
            print('iter: ',i,new_policy)
        random_policy = new_policy
    return new_policy

In [38]:
optimal_policy = policy_iteration(env,gamma=1.0)

iter:  0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Policy-Iteration converged at step 7.


In [39]:
print(optimal_policy)

[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]


In [40]:
env.reset()
max_steps = 100
# lets test for 10 episodes
for episode in range(10):
    state = env.reset();
    step = 0;
    done = False;
    print("XXXXXXXXXX");
    print("Episode: ",episode);
    
    for step in range(max_steps):
        
        #take the action with maximum expected future reward form the q-table
        #action = np.argmax(q_table[state,:]);
        action = int(optimal_policy[state])
        
        new_state, reward, done, info = env.step(action);
        
        if done:
            #print only the last stage to check if the agent reached the goal(G) or fell into a hole(H)
            env.render();
            
            print("Number of steps taken: ",step);
            break;
            
        state = new_state;

#close the connection to the environment
env.close();

XXXXXXXXXX
Episode:  0
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  91
XXXXXXXXXX
Episode:  1
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  81
XXXXXXXXXX
Episode:  2
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  19
XXXXXXXXXX
Episode:  3
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  14
XXXXXXXXXX
Episode:  4
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  23
XXXXXXXXXX
Episode:  5
  (Up)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  99
XXXXXXXXXX
Episode:  6
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  24
XXXXXXXXXX
Episode:  7
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  62
XXXXXXXXXX
Episode:  8
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  72
XXXXXXXXXX
Episode:  9
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps taken:  53


In [17]:
# frozen-lake-ex2.py
import gym
 
env = gym.make('FrozenLake-v0',  is_slippery=False)

ans = [1,1,2,2,1,2]
state = env.reset()
print("Start State: ", state)
for i in ans:
    a,b,c,d = env.step(i)
    env.render()
env.close()

Start State:  0
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
